## Select best dammit annotations for a trinity transcriptome. Create tx2gene maps for use with tximport; aggregate contigs with identical "best" annotations

In [1]:
import pandas as pd
# requires dammit env (mine-->source activate dammit), or dammit installed in main env
# if using an env: if you haven't done so yet, first install an ipy kernel in dammit env:
# ipython kernel install --user --name dammit
# then start new notebook using "dammit" kernel, or for existing nb, select "switch kernel" in the Kernel menu
# voila! :D
from dammit.fileio.gff3 import GFF3Parser

#### Link Dammit and Trinity names
Use Dammit Namemap to create Dammit_Name : Trinity Transcript/Gene map

In [315]:
namemap = "Trinity.fasta.dammit.namemap.csv"
trin2dammit = pd.read_csv(namemap)

# create gene: trans map while we're at it
trin2dammit['Trinity_transcript'] = trin2dammit['original'].str.split(' ', 1).str[0]
trin2dammit['Trinity_gene'] = trin2dammit['Trinity_transcript'].str.rsplit('_i', 1).str[0] 

# drop 'original' trinity name; rename dammit name column
trin2dammit.rename(index=str, columns={"renamed": "Dammit_transcript"}, inplace=True)
trin2dammit.drop(columns=['original'], inplace=True)

#print(trin2dammit.shape)
#trin2dammit.head()

#### Parse Dammit GFF3
Use Dammit GFF3 to read in annotations, and link with Trinity Names

In [133]:
annots_gff3_file = "Trinity.fasta.dammit.gff3"
annots = GFF3Parser(filename=annots_gff3_file).read() # read in annotation gff3
#print(annots.shape)
#annots.head()

/Users/tessa/anaconda3/envs/dammit/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


In [134]:
# merge annotation gff3 with dammit-trinity namemap
annotsTrin = pd.merge(trin2dammit, annots, how='outer', left_on="Dammit_transcript", right_on="seqid")
#print(annotsTrin.shape)
#annotsTrin.head()

In [132]:
#count transcripts/annotations & do a sanity check on the merge:
numTranscripts = trin2dammit['Dammit_transcript'].nunique()
numWithAnnots = annots['seqid'].nunique()
numNoAnnots = numTranscripts - numWithAnnots
numMergedTranscripts = annotsTrin['Dammit_transcript'].nunique()

if (numMergedTranscripts != numTranscripts):
    print('something went wrong during merge')
else:
    print('Of ' + str(numTranscripts) + ' total transcripts:')
    print('    ' + str(numWithAnnots) + ' transcripts have at least one annotation')
    print('    ' + str(numNoAnnots)  + ' transcripts have no annotations')

Of 150663 total transcripts:
    63270 transcripts have at least one annotation
    87393 transcripts have no annotations


#### Select annotations with best e-values

Create annotation csv with the 1. best e-val hit, and 2. best Obimac hit for each transcript

In [153]:
annotsTrin.head()

,Dammit_transcript,Trinity_transcript,Trinity_gene,Dbxref,ID,Name,Note,Parent,Target,accuracy,...,end,env_coords,phase,score,seqid,source,start,strand,trunc,type
0,Transcript_0,TRINITY_DN49110_c0_g1_i1,TRINITY_DN49110_c0_g1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Transcript_1,TRINITY_DN49156_c0_g1_i1,TRINITY_DN49156_c0_g1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Transcript_2,TRINITY_DN49121_c0_g1_i1,TRINITY_DN49121_c0_g1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Transcript_3,TRINITY_DN49116_c0_g1_i1,TRINITY_DN49116_c0_g1,NaN,cds.Transcript_3.p2,NaN,NaN,Transcript_3.p2,NaN,NaN,...,482.0,NaN,0.0,NaN,Transcript_3,transdecoder,173.0,+,NaN,CDS
4,Transcript_3,TRINITY_DN49116_c0_g1_i1,TRINITY_DN49116_c0_g1,NaN,Transcript_3.p2.exon1,NaN,NaN,Transcript_3.p2,NaN,NaN,...,482.0,NaN,NaN,NaN,Transcript_3,transdecoder,0.0,+,NaN,exon


In [404]:
# find best Obimac hit per transcript (using best evalue)
Obimac = annotsTrin[annotsTrin['database'] == "Obimac_refseq_protein.fa.gz"]
Obimac = Obimac.sort_values(by=['Trinity_transcript', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='Trinity_transcript')[['Trinity_transcript','Name', 'score', 'start', 'end']]
#Obimac = Obimac.dropna(axis=0,how="all")
Obimac.rename(index=str, columns={'Name': 'Obimac_Name', 'score': 'Obimac_score', 'start': 'Obimac_start', 'end':'Obimac_end'}, inplace=True)

Obimac.tail()

#print(Obimac.shape)
#Obimac.head()
#list(Obimac['Name']) # if you want to read the names
# find best Eval that is *not* an Obimac hit

noObimac = annotsTrin[annotsTrin['database'] != 'Obimac_refseq_protein.fa.gz']
bestEval = noObimac.sort_values(by=['Trinity_transcript', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='Trinity_transcript') #[['Dammit_transcript','Name', 'score', 'start', 'end','database']]
#bestEval = bestEval.dropna(axis=0,how="all")

print(bestEval.shape)
#bestEval.head()
# merge bestEval and Obimac annotation info
bestAnnots = pd.merge(bestEval, Obimac, how='outer', left_on="Trinity_transcript", right_on="Trinity_transcript")
print(bestAnnots.shape)

# to keep the unannotated transcripts, merge those back in
bestAnnotFull = pd.merge(bestAnnots, trin2dammit[['Trinity_transcript']], how='outer', left_on=["Trinity_transcript"],right_on=["Trinity_transcript"])
print(bestAnnotFull.shape)
bestAnnotFull.to_csv("dammit_bestEvalperTranscript.csv", index=False, sep='\t')
bestEval.to_csv("bestEvalpertranscript.csv")

(44964, 22)
(46472, 26)
(150663, 26)


The above keeps all identified trinity transcripts, and just reports the best annotation (determined by eval), as well as the best Obimac annotation (also det. by eval).

However, there are undoubtedly some duplicate annotations, as there are v. likely not that many genes in D. opalescens. For differential expression, we're primarily interested in functional changes, and spreading the expression of a gene across different contigs with the exact same best annotation is not desirable. Can we collapse by best annotation?

#### Create Tx2Gene files
Create Transcript to Gene (tx2gene) files that assign transcripts to their annotated gene names,
rather than just the Trinity gene name. If we want to keep unannotated transcripts, we can retain 
Trinity gene name for unannotated transcripts. These can be used with tximport to aggregate transcript-level
quantification to gene-level counts for differential expression analysis.

In [405]:
bestAnnot_tx2gene = bestAnnots[['Trinity_transcript', 'Name']].dropna()
#bestAnnots['Name'].nunique() # just checking that there are duplicate 'Name' column values. There are.
bestAnnot_tx2gene.head()
bestAnnot_tx2gene.to_csv('bestEval_tx2gene.txt', index=False, sep = '\t')
# do this using the Obimac best annotations instead:
bestObimac_tx2gene = bestAnnots[['Trinity_transcript', 'Obimac_Name']].dropna()
bestObimac_tx2gene.to_csv('bestObimac_tx2gene.txt', index=False, sep = '\t') 

Let's make a version that includes unannotated transcripts. Use Trinity gene info as gene name

In [406]:
#remake df: (easier for testing)
bestAnnotFull = pd.merge(bestAnnots, trin2dammit[['Trinity_transcript']], how='outer', left_on=["Trinity_transcript"],right_on=["Trinity_transcript"])
print(bestAnnotFull.shape)
# first, fill NA in the 'Name' column with Obimac annotations
numAnnots = len(list(bestAnnotFull.Name.dropna()))
print('# annotations using only best Evalue: ' + str(numAnnots))
bestAnnotFull.Name = bestAnnotFull.Name.fillna(value=bestAnnotFull.Obimac_Name)
numAnnots_wOb = len(list(bestAnnotFull.Name.dropna()))
print('# annotations using best Evalue AND best Obimac Evalue: ' + str(numAnnots_wOb))
bestEandObimac_tx2gene = bestAnnotFull[['Trinity_transcript', 'Name']].dropna()
bestEandObimac_tx2gene.to_csv('bestEval_Obimac_tx2gene.txt', index=False, sep = '\t')

(150663, 26)
# annotations using only best Evalue: 44964
# annotations using best Evalue AND best Obimac Evalue: 46472


In [409]:
# Then, fill remaining NA values in the 'Name' column with the Trinity Gene name:
bestAnnotFull.Name = bestAnnotFull.Name.fillna(value=bestAnnotFull.Trinity_gene)
bestAnnotFull_tx2gene = bestAnnotFull[['Trinity_transcript', 'Name']]
print(bestAnnotFull_tx2gene.shape)
bestAnnotFull_tx2gene.to_csv('bestEvalFull_tx2gene.txt', index=False, sep = '\t')
bestAnnotFull.head()

(150663, 2)


,Dammit_transcript,Trinity_transcript,Trinity_gene,Dbxref,ID,Name,Note,Parent,Target,accuracy,...,seqid,source,start,strand,trunc,type,Obimac_Name,Obimac_score,Obimac_start,Obimac_end
0,Transcript_55709,TRINITY_DN10002_c0_g1_i1,TRINITY_DN10002_c0_g1,NaN,homology:127783,LotgiP220800,NaN,NaN,LotgiP220800 1222 1355 +,NaN,...,Transcript_55709,LAST,1.0,-,NaN,translated_nucleotide_match,NaN,NaN,NaN,NaN
1,Transcript_55702,TRINITY_DN10017_c0_g1_i1,TRINITY_DN10017_c0_g1,"""Pfam:PF00085.16""",homology:32447,Thioredoxin,Thioredoxin,NaN,Thioredoxin 5 74 +,0.92,...,Transcript_55702,HMMER,141.0,NaN,NaN,protein_hmm_match,NaN,NaN,NaN,NaN
2,Transcript_55699,TRINITY_DN10022_c0_g1_i1,TRINITY_DN10022_c0_g1,NaN,homology:127777,SMAR010045-PA,NaN,NaN,SMAR010045-PA 58 105 +,NaN,...,Transcript_55699,LAST,403.0,-,NaN,translated_nucleotide_match,NaN,NaN,NaN,NaN
3,Transcript_55695,TRINITY_DN10025_c0_g1_i1,TRINITY_DN10025_c0_g1,NaN,homology:127774,OABI005503-RA,NaN,NaN,OABI005503-RA 401 466 +,NaN,...,Transcript_55695,LAST,514.0,-,NaN,translated_nucleotide_match,gi|961083531|ref|XP_014769378.1| PREDICTED: BR...,3.400000e-55,27.0,236.0
4,Transcript_55697,TRINITY_DN10032_c0_g1_i1,TRINITY_DN10032_c0_g1,NaN,homology:127776,CapteP182594,NaN,NaN,CapteP182594 1098 1233 +,NaN,...,Transcript_55697,LAST,14.0,-,NaN,translated_nucleotide_match,gi|961119188|ref|XP_014781878.1| PREDICTED: ca...,4.200000e-70,0.0,129.0


In [ ]:
#Not used: Aggregate by identical best eval hits:
# Collapse using best Eval hit:
#bestAnnots.groupby('Name',as_index=False)[['Dammit_transcript']].aggregate(lambda x: list(x))
#Obimac.groupby('Name',as_index=False)[['seqid']].aggregate(lambda x: list(x))
# this groups by Name, but it it necessary for what I want? I just need a transcript \t gene, where transcript 
# is the trinity transcript name, and 'gene' is the 'Name' value.